# Dog Breed Classification



In [2]:
!pip install smdebug

  Using cached smdebug-1.0.34-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp310-cp310-linux_x86_64.whl
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached smdebug-1.0.34-py2.py3-none-any.whl (280 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires to

In [3]:
import sagemaker
import boto3
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner, IntegerParameter
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.debugger import Rule, rule_configs, ProfilerRule, DebuggerHookConfig, CollectionConfig, ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

print(bucket)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
[2024-02-15 10:59:34.421 sagemaker-data-scienc-ml-t3-medium-ccb588b5efaf671be41927273f0c:20 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker-us-east-1-283573335284


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#Fetch and upload the data to AWS S3
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip;
!unzip dogImages.zip;

--2024-02-15 10:59:53--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.112.248, 52.219.112.80, 52.219.193.152, ...
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.112.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip.1’

dogImages.zip.1     100%[===================>]   1.05G  47.8MB/s    in 41s     

2024-02-15 11:00:35 (26.6 MB/s) - ‘dogImages.zip.1’ saved [1132023110/1132023110]

Archive:  dogImages.zip
replace dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
# copy data to s3 bucket
inputs = sagemaker_session.upload_data(path="dogImages", bucket=bucket, key_prefix='imagesDogClassesSet')

In [6]:
inputs

's3://sagemaker-us-east-1-283573335284/imagesDogClassesSet'

In [7]:
# varify if data has been uploaded to s3
!aws s3 ls s3://${bucket}/


An error occurred (NoSuchBucket) when calling the ListObjectsV2 operation: The specified bucket does not exist


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [4]:
#Load necessary packages
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [8]:
hyperparameter_ranges = {
    'lr': ContinuousParameter(0.001, 0.1),
    'batch_size': CategoricalParameter([16 ,32, 64]),
    'epochs': IntegerParameter(2, 6)
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [9]:
estimator = PyTorch(entry_point = 'hpo.py',
                    base_job_name = 'dog-breed-classification-hpo',
                    role = role,
                    instance_count = 1,
                    instance_type = 'ml.m5.2xlarge', 
                    py_version = 'py36',
                    framework_version = '1.8'
                   )

tuner = HyperparameterTuner(estimator, 
                            objective_metric_name, 
                            hyperparameter_ranges, 
                            metric_definitions, 
                            max_jobs = 4, 
                            max_parallel_jobs = 1, 
                            objective_type = objective_type
                           )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [10]:
tuner.fit({'train': inputs}, wait=True)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Using provided s3_resource
...............................................................................................................................................................................................................................................................................!


In [11]:
# Get the best estimators and the best HPs
tuner.best_estimator().hyperparameters()


2024-02-15 11:33:54 Starting - Found matching resource for reuse
2024-02-15 11:33:54 Downloading - Downloading the training image
2024-02-15 11:33:54 Training - Training image download completed. Training in progress.
2024-02-15 11:33:54 Uploading - Uploading generated training model
2024-02-15 11:33:54 Completed - Resource reused by training job: pytorch-training-240215-1124-003-ec57b458


{'_tuning_objective_metric': '"average test loss"',
 'batch_size': '"64"',
 'epochs': '4',
 'lr': '0.0030844061911579216',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"dog-breed-classification-hpo-2024-02-15-11-24-19-952"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-283573335284/dog-breed-classification-hpo-2024-02-15-11-24-19-952/source/sourcedir.tar.gz"'}

In [12]:
#Get the hyperparameters of the best trained model
best_hyperparameters = {'batch_size': 64, 'epochs': 2, 'lr':0.07927295703989701}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [13]:
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

profiler_config = ProfilerConfig(system_monitor_interval_millis=500, 
                                                       framework_profile_params = FrameworkProfile(num_steps=10))

collection_config = [CollectionConfig(name="CrossEntropyLoss_output_0", 
                                    parameters={"include_regex": "CrossEntropyLoss_output_0",
                                                            "train.save_interval": "10",
                                                            "eval.save_interval": "1"})]

debugger_config = DebuggerHookConfig(collection_configs = collection_config)

Framework profiling will be deprecated from tensorflow 2.12 and pytorch 2.0 in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
estimator = PyTorch(entry_point = 'train_model.py',
                                    role = role,
                                    instance_count=1,
                                    instance_type = 'ml.m5.2xlarge',
                                    py_version = 'py36',
                                    framework_version = '1.6',
                                    hyperparameters = best_hyperparameters,
                                    profiler_config = profiler_config, 
                                    debugger_hook_config = debugger_config, 
                                    rules = rules ) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [17]:
estimator.fit({'train':inputs}, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-02-15-12-00-11-940


Using provided s3_resource
2024-02-15 12:00:12 Starting - Starting the training job...
2024-02-15 12:00:40 Starting - Preparing the instances for trainingVanishingGradient: InProgress
PoorWeightInitialization: InProgress
ProfilerReport: InProgress
......
2024-02-15 12:01:40 Downloading - Downloading input data......
2024-02-15 12:02:42 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-02-15 12:02:54,641 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-02-15 12:02:54,643 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-02-15 12:02:54,654 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-02-15 12:02:54,656 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-02-15 12:02:54,832 sagemaker-training-to

In [13]:
job_name = estimator.latest_training_job.name
client = estimator.sagemaker_session.sagemaker_client
description = client.describe_training_job(TrainingJobName=estimator.latest_training_job.name)

print(f"Jobname: {job_name}")
print(f"Client: {client}")
print(f"Description: {description}")

Jobname: pytorch-training-2024-02-07-14-39-35-871
Client: <botocore.client.SageMaker object at 0x7fe3782bde10>
Description: {'TrainingJobName': 'pytorch-training-2024-02-07-14-39-35-871', 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:283573335284:training-job/pytorch-training-2024-02-07-14-39-35-871', 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/output/model.tar.gz'}, 'TrainingJobStatus': 'Completed', 'SecondaryStatus': 'Completed', 'HyperParameters': {'batch_size': '64', 'epochs': '2', 'lr': '0.07927295703989701', 'sagemaker_container_log_level': '20', 'sagemaker_job_name': '"pytorch-training-2024-02-07-14-39-35-871"', 'sagemaker_program': '"train_model.py"', 'sagemaker_region': '"us-east-1"', 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/source/sourcedir.tar.gz"'}, 'AlgorithmSpecification': {'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazon

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [14]:
# TODO: Display the profiler output
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print('Profiler report location: {}'.format(rule_output_path))

Profiler report location: s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/rule-output


In [15]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print('Profiler report location: {}'.format(rule_output_path))

Profiler report location: s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/rule-output


In [16]:
import os

profiler_report_name = [ rule["RuleConfigurationName"]
                                            for rule in estimator.latest_training_job.rule_job_summary()
                                            if "Profiler" in rule["RuleConfigurationName"]
                                            ][0]

## Model Deploying

In [21]:
model_location = estimator.model_data
model_location

's3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/output/model.tar.gz'

In [20]:
# TODO: Deploy your model to an endpoint
predictor=estimator.deploy(initial_instance_count=1, instance_type="ml.m5.2xlarge") # TODO: Add your deployment configuration like instance type and number of instances

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/output/model.tar.gz), script artifact (s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-39-35-871/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-283573335284/pytorch-training-2024-02-07-14-47-04-778/model.tar.gz. This may take some time depending on model size...


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()